### Generate JSONL files from CSV files with original TLDR format
Original TLDR format: Contains a list of target tldrs per sample for test and dev/valid sets

In [1]:
# Imports
import os
import json
import pandas as pd

In [2]:
# Read in csv data

def read_csv_file(file_name):
    data = pd.read_csv(file_name, encoding="utf-8")
    return data.to_dict()

def read_jsonl_files(file_name):
    data = []
    file = open(file_name, "r", encoding="utf-8")
    lines = file.readlines()
    for line in lines:
        data.append(json.loads(line))
    return data

def generate_jsonl(json_data, csv_data, output_file):
    data = []
    for i in range(len(json_data)):
        content = {}
        content["source"] = json_data[i]["source"]
        content["source_labels"] = json_data[i]["source_labels"] #str() ?
        content["rouge_scores"] = []
        content["paper_id"] = json_data[i]["paper_id"]
        target = []
        for j in range(len(csv_data["paper_id"])):
            if (json_data[i]["paper_id"] == csv_data["paper_id"][j]):
                target.append(csv_data["target"][j])
        content["target"] = target
        data.append(content)
    file = open(f"{output_file}.jsonl", "w", encoding="utf-8")
    for ele in data:
        file.write(str(json.dumps(ele)) + "\n")
    file.close()
    

In [3]:
###

In [5]:
## Generate Language specific files
# languages = [["French", "fr"], ["German", "de"], ["Italian", "it"], ["Japanese", "ja"], ["Spanish", "es"]]"
languages = [["Russian", "ru"]]
versions = ["Abstracts", "AIC", "FullText"]
files_en = ["train", "test", "dev"]
files_tgt = ["train", "test", "valid"]

for lang in languages:
    for version in versions:
        for k in range(len(files_tgt)):
            json_data = read_jsonl_files(os.path.join("English", "jsonl", version, f"{files_en[k]}.jsonl"))
            csv_data = read_csv_file(os.path.join(lang[0], "csv", version, f"{files_tgt[k]}-{lang[1]}.csv"))
            generate_jsonl(json_data, csv_data, os.path.join(lang[0], "jsonl", version, f"{files_tgt[k]}-{lang[1]}"))